In [1]:
import torch.cuda

from src import  N_CLASSES, FEATURE_DIM

BATCH_SIZE = 32
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'device: {device}')

device: cpu


In [2]:
# 读数据
data = torch.load("data.pth")
feature = data['feature']
label = data['label']

In [3]:
# 生成训练集、测试集
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, train_size=0.8, random_state=0)
x_train, x_test, y_train, y_test = x_train.to(device), x_test.to(device), y_train.to(device), y_test.to(device)

In [4]:
# 加载数据
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# 定义网络
import torch.nn as nn


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden = nn.Linear(FEATURE_DIM, FEATURE_DIM)
        self.out = nn.Linear(FEATURE_DIM, N_CLASSES)

    def forward(self, x):
        x = torch.sigmoid(self.hidden(x))
        x = self.out(x)
        return x


net = Net().to(device)

# 定义损失函数、优化器
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),
                       lr=0.001,
                       betas=(0.9, 0.999),
                       eps=1e-08,
                       weight_decay=0,
                       amsgrad=False)

In [6]:
# 训练网络
for epoch in range(1000):  # 数字越大，循环次数越多
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    if epoch % 10 == 0:
        print(f'epoch {epoch} loss: {running_loss / 1500:.10f}')
        running_loss = 0.0

print('Finished Training')

# 训练结束后保存网络参数
PATH = './model.pth'
torch.save(net.state_dict(), PATH)

epoch 0 loss: 0.3291572724
epoch 10 loss: 0.0647634281
epoch 20 loss: 0.0449954403
epoch 30 loss: 0.0303180392
epoch 40 loss: 0.0258456737
epoch 50 loss: 0.0245770348
epoch 60 loss: 0.0239587211
epoch 70 loss: 0.0176149821
epoch 80 loss: 0.0132854770
epoch 90 loss: 0.0139324281
epoch 100 loss: 0.0151357625
epoch 110 loss: 0.0118461758
epoch 120 loss: 0.0116393077
epoch 130 loss: 0.0082939467
epoch 140 loss: 0.0100089271
epoch 150 loss: 0.0094873867
epoch 160 loss: 0.0089448032
epoch 170 loss: 0.0066554623
epoch 180 loss: 0.0059360032
epoch 190 loss: 0.0066568274
epoch 200 loss: 0.0060255331
epoch 210 loss: 0.0056815776
epoch 220 loss: 0.0095185431
epoch 230 loss: 0.0050715698
epoch 240 loss: 0.0032584682
epoch 250 loss: 0.0080648881
epoch 260 loss: 0.0049500790
epoch 270 loss: 0.0043488823
epoch 280 loss: 0.0045837761
epoch 290 loss: 0.0040580846
epoch 300 loss: 0.0042817925
epoch 310 loss: 0.0029149554
epoch 320 loss: 0.0036762658
epoch 330 loss: 0.0039788055
epoch 340 loss: 0.0034219

In [7]:
# 测试模型
net = Net().to(device)
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    # noinspection PyShadowingBuiltins
    for input, labels in test_loader:
        # calculate outputs by running images through the network
        outputs = net(input)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # noinspection PyUnresolvedReferences
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total}')

Accuracy: 0.9713333333333334
